In [12]:
import websockets
import asyncio
import json

In [5]:
async def bithumb_ws_client():
    uri = 'wss://pubwss.bithumb.com/pub/ws'
    
    async with websockets.connect(uri) as websocket:
        greeting = await websocket.recv()
        print(greeting)

async def main():
    await bithumb_ws_client()

In [14]:
uri = 'wss://pubwss.bithumb.com/pub/ws'
    
async with websockets.connect(uri) as websocket:
    greeting = await websocket.recv()
    print(greeting)
    
    subscribe_fmt = {
            "type":"ticker", 
            "symbols": ["BTC_KRW"], 
            "tickTypes": ["1H"]
    }
    subscribe_data = json.dumps(subscribe_fmt)
    await websocket.send(subscribe_data)
    
    while True:
        data = await websocket.recv()
        data = json.loads(data)
        print(data)

{"status":"0000","resmsg":"Connected Successfully"}
{'status': '0000', 'resmsg': 'Filter Registered Successfully'}
{'type': 'ticker', 'content': {'tickType': '1H', 'date': '20220311', 'time': '135101', 'openPrice': '48129000', 'closePrice': '48062000', 'lowPrice': '48007000', 'highPrice': '48632000', 'value': '7588408161.18417', 'volume': '157.48406635', 'sellVolume': '65.24400772', 'buyVolume': '92.24005863', 'prevClosePrice': '48760000', 'chgRate': '-0.14', 'chgAmt': '-67000', 'volumePower': '141.38', 'symbol': 'BTC_KRW'}}
{'type': 'ticker', 'content': {'tickType': '1H', 'date': '20220311', 'time': '135103', 'openPrice': '48135000', 'closePrice': '48061000', 'lowPrice': '48007000', 'highPrice': '48632000', 'value': '7590101831.78417', 'volume': '157.51946635', 'sellVolume': '65.39370772', 'buyVolume': '92.12575863', 'prevClosePrice': '48760000', 'chgRate': '-0.15', 'chgAmt': '-74000', 'volumePower': '140.88', 'symbol': 'BTC_KRW'}}
{'type': 'ticker', 'content': {'tickType': '1H', 'dat

CancelledError: 

In [15]:
uri = 'wss://pubwss.bithumb.com/pub/ws'
    
async with websockets.connect(uri) as websocket:
    greeting = await websocket.recv()
    print(greeting)
    
    subscribe_fmt = {
            "type":"transaction", 
            "symbols": ["BTC_KRW"], 
            "tickTypes": ["1H"]
    }
    subscribe_data = json.dumps(subscribe_fmt)
    await websocket.send(subscribe_data)
    
    while True:
        data = await websocket.recv()
        data = json.loads(data)
        print(data)

{"status":"0000","resmsg":"Connected Successfully"}
{'status': '0000', 'resmsg': 'Filter Registered Successfully'}
{'type': 'transaction', 'content': {'list': [{'buySellGb': '2', 'contPrice': '48302000', 'contQty': '0.107', 'contAmt': '5168314.000', 'contDtm': '2022-03-11 15:39:21.543194', 'updn': 'up', 'symbol': 'BTC_KRW'}]}}
{'type': 'transaction', 'content': {'list': [{'buySellGb': '2', 'contPrice': '48280000', 'contQty': '0.0857', 'contAmt': '4137596.0000', 'contDtm': '2022-03-11 15:39:29.741684', 'updn': 'dn', 'symbol': 'BTC_KRW'}]}}
{'type': 'transaction', 'content': {'list': [{'buySellGb': '2', 'contPrice': '48280000', 'contQty': '0.0564', 'contAmt': '2722992.0000', 'contDtm': '2022-03-11 15:39:29.860322', 'updn': 'dn', 'symbol': 'BTC_KRW'}]}}
{'type': 'transaction', 'content': {'list': [{'buySellGb': '2', 'contPrice': '48279000', 'contQty': '0.0931', 'contAmt': '4494774.9000', 'contDtm': '2022-03-11 15:39:30.853592', 'updn': 'dn', 'symbol': 'BTC_KRW'}, {'buySellGb': '2', 'contP

ConnectionClosedError: sent 1011 (unexpected error) keepalive ping timeout; no close frame received

In [29]:
uri = 'wss://api.upbit.com/websocket/v1'
    
async with websockets.connect(uri) as websocket:
    s_fmt = [{"ticket":"test"},{"type":"ticker","codes":["KRW-BTC"]}]
    subscribe_fmt = [
        {"ticket": "sample"},
        {
            "type":"ticker", 
            "codes": ["KRW_BTC", "KRW_ETH"], 
            "isOnlySnapshot": True
            #"isOnlyRealtime": True
        }
        #, {"format": "SIMPLE"}
    ]
    subscribe_data = json.dumps(s_fmt)
    await websocket.send(subscribe_data)
    
    while True:
        data = await websocket.recv()
        data = json.loads(data)
        print(data)

{'type': 'ticker', 'code': 'KRW-BTC', 'opening_price': 48931000.0, 'high_price': 49941000.0, 'low_price': 47802000.0, 'trade_price': 49606000.0, 'prev_closing_price': 48917000.0, 'acc_trade_price': 206057547543.75616, 'change': 'RISE', 'change_price': 689000.0, 'signed_change_price': 689000.0, 'change_rate': 0.0140850829, 'signed_change_rate': 0.0140850829, 'ask_bid': 'BID', 'trade_volume': 0.004, 'acc_trade_volume': 4232.45587348, 'trade_date': '20220311', 'trade_time': '121307', 'trade_timestamp': 1647000787000, 'acc_ask_volume': 2212.29062788, 'acc_bid_volume': 2020.1652456, 'highest_52_week_price': 82700000.0, 'highest_52_week_date': '2021-11-09', 'lowest_52_week_price': 33900000.0, 'lowest_52_week_date': '2021-06-22', 'market_state': 'ACTIVE', 'is_trading_suspended': False, 'delisting_date': None, 'market_warning': 'NONE', 'timestamp': 1647000787015, 'acc_trade_price_24h': 375165973870.6162, 'acc_trade_volume_24h': 7692.03789885, 'stream_type': 'SNAPSHOT'}
{'type': 'ticker', 'code

{'type': 'ticker', 'code': 'KRW-BTC', 'opening_price': 48931000.0, 'high_price': 49941000.0, 'low_price': 47802000.0, 'trade_price': 49612000.0, 'prev_closing_price': 48917000.0, 'acc_trade_price': 206099955527.58255, 'change': 'RISE', 'change_price': 695000.0, 'signed_change_price': 695000.0, 'change_rate': 0.0142077396, 'signed_change_rate': 0.0142077396, 'ask_bid': 'ASK', 'trade_volume': 0.00817272, 'acc_trade_volume': 4233.31072963, 'trade_date': '20220311', 'trade_time': '121309', 'trade_timestamp': 1647000789000, 'acc_ask_volume': 2212.2988006, 'acc_bid_volume': 2021.01192903, 'highest_52_week_price': 82700000.0, 'highest_52_week_date': '2021-11-09', 'lowest_52_week_price': 33900000.0, 'lowest_52_week_date': '2021-06-22', 'market_state': 'ACTIVE', 'is_trading_suspended': False, 'delisting_date': None, 'market_warning': 'NONE', 'timestamp': 1647000788790, 'acc_trade_price_24h': 375165973870.6162, 'acc_trade_volume_24h': 7692.03789885, 'stream_type': 'REALTIME'}
{'type': 'ticker', 

CancelledError: 

In [21]:
!telsocket -url wss://api.upbit.com/websocket/v1

'telsocket'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
